Install Dependencies

In [55]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

Selects the dataframe for training

In [92]:
#Downloads dataframe from internet

df = yf.download(tickers="ETH-USD",
                            period="60d",
                            interval="15m",
                            auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [67]:

#Loads dataframe from folder

#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
#df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_1hr.csv')
#df = pd.read_csv('BTCUSD_day.csv')
#df = pd.read_csv('BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')


Data Cleanup

In [87]:
#print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
#df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
#df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)


#Preparation for hourly df
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
#df = df.drop(columns=['Volume'])

#Preparation for downloaded df

df = df.reset_index(drop=True)
df = df.drop(columns=['Volume'])

'''
#Preparation for daily dataset
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})
'''

print(df.shape[0])
print(df.dtypes)

df.head()

1435
Open     float64
High     float64
Low      float64
Close    float64
dtype: object


,Open,High,Low,Close
0,1200.153076,1205.530151,1200.153076,1204.813110
1,1208.716797,1214.011719,1205.333862,1214.011719
2,1214.143066,1220.330078,1213.272339,1218.716919
3,1218.397949,1221.807129,1217.581421,1219.858154
4,1219.493652,1223.531616,1218.365967,1222.480469


2.1.3. Calculate SMA, RSI and OBV


In [88]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,High,Low,Close,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
0,1200.153076,1205.530151,1200.153076,1204.813110,1200.153076,1200.153076,1200.153076,0.000000,0.000000,0.000000,1204.813110,0.000000,100.000000,0.000000
1,1208.716797,1214.011719,1205.333862,1214.011719,1204.434937,1204.434937,1204.434937,0.000000,0.000000,0.000000,1214.011719,0.733792,100.000000,0.000000
2,1214.143066,1220.330078,1213.272339,1218.716919,1207.670980,1207.670980,1207.670980,0.000000,0.000000,0.000000,1204.813110,1.675682,100.000000,0.000000
3,1218.397949,1221.807129,1217.581421,1219.858154,1210.352722,1210.352722,1210.352722,0.000000,0.000000,0.000000,1213.272339,2.485572,100.000000,0.000000
4,1219.493652,1223.531616,1218.365967,1222.480469,1212.180908,1212.180908,1212.180908,0.000000,0.000000,0.000000,1213.272339,3.300964,100.000000,0.000000
5,1222.756226,1222.756226,1215.711792,1216.759521,1213.943461,1213.943461,1213.943461,0.000000,0.000000,0.000000,1213.640664,3.445813,71.027427,0.000000
6,1215.834229,1218.848755,1214.693970,1218.848755,1214.213571,1214.213571,1214.213571,0.000000,0.000000,0.000000,1213.994256,3.686693,73.990993,0.000000
7,1220.198608,1229.548828,1220.198608,1226.297363,1217.077218,1214.961700,1214.961700,0.000000,0.000000,0.000000,1214.333705,4.427595,81.325238,0.000000
8,1227.412598,1230.221191,1225.823853,1230.221191,1219.748047,1216.345133,1216.345133,0.000000,0.000000,0.000000,1214.693970,5.270628,83.900715,0.000000
9,1231.236328,1233.134277,1229.381714,1232.998535,1222.189941,1217.834253,1217.834253,0.000000,0.000000,0.000000,1215.936147,6.092614,85.432163,0.000000


In [89]:
#add a column to dataframe
df.drop(df.tail(1).index,inplace=True)
df['Position'] = np.where((df['Open']-df['Close'].shift(-1)) < 1, 1, 0)

#df["Position"] = df['Open'] - df['Close'].shift(-1)



In [90]:
df = df.reset_index(drop=True)


Dump dataset into a pickle file

In [91]:
import pickle
with open("ETHUSD_15min_tagged.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
